# TCLab_CLP_PID_FF

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from datetime import datetime
import tclab
import time

import os

from matplotlib import colors as mcolors

import package_DBR
from package_DBR import myRound, SelectPath_RT, Delay_RT, FO_RT, FOPDT, SOPDT, FOPDT_cost, SOPDT_cost, Process, Bode

import package_LAB
from package_LAB import LeadLag_RT, PID_RT, IMCTuning

from importlib import reload
package_DBR = reload(package_DBR)
package_LAB = reload(package_LAB)

In [15]:
TSim = 2000
Ts = 1
N = int(TSim/Ts) + 1

In [16]:
# Working point
MV0 = 50
DV0 = 50
PV0 = 55

In [17]:
#SOPDT from PV
Kp = 0.398981565560627
T1p = 165.56685644081972
T2p = 3.645364787323084e-05
thetap = 0.6102455489040157

In [18]:
# from DV
Kd = 0.31148119513939204
T1d = 167.0472401651847
T2d = 14.406792795634894
thetad = 9.375122302137418


In [19]:
gamma = 0.4 # Lower = more aggressive
Kc,Ti,Td = IMCTuning(Kp,T1p,T2p,thetap,gamma,'SOPDT')

In [20]:
alpha = 1
MVMin = 0
MVMax = 100

In [12]:
settings = 'Man'

if settings == 'Man':
    #  OLP + no FF
    ManPath = {0: True,TSim: True} 
    MVManPath = {0: MV0, TSim: MV0} 
    SPPath = {0: PV0, TSim: PV0} 
    DVPath = {0: 50.0, 1000: 60, TSim: 60.0} 
    ManFF = False
    Title_loop = "Open-loop"
    Title_FF = "no feedforward"
    
elif settings == 'ManFF':
    # OLP + FF
    ManPath = {0: True,TSim: True}
    MVManPath = {0: MV0, TSim: MV0} 
    SPPath = {0: PV0, TSim: PV0} 
    DVPath = {0: 50.0, 1000: 60, TSim: 60.0}
    ManFF = True
    Title_loop = "Open-loop"
    Title_FF = "feedforward"

elif settings == 'FF':
    # FF, CLP
    ManPath = {0: True, 750: False, TSim: False}
    MVManPath = {0: MV0, TSim: MV0}
    SPPath = {0: PV0, 800: PV0-10, TSim: PV0-10} 
    DVPath = {0: DV0, 1500: DV0+10, TSim: DV0+10}
    ManFF = True
    Title_loop = "Closed-loop"
    Title_FF = "feedforward"
    
else:
    # no FF, CLP
    ManPath = {0: True, 750: False, TSim: False} 
    MVManPath = {0: MV0, TSim: MV0} 
    SPPath = {0: PV0, 800: PV0-10, TSim: PV0-10} 
    DVPath = {0: DV0, 1500: DV0+10, TSim: DV0+10} 
    ManFF = False
    Title_loop = "Closed-loop"
    Title_FF = "no feedforward"


In [22]:
t = []
SP = []
MV = []
Man = []
MVMan = []
MVFF = []
MVP = []
MVI = []
MVD = []
DV = []
PV = []
E = []

MVFFDelay = []
MVFFLL1 = []

MVDelayp = []
PV1p = []
PV2p = []

MVDelayd = []
PV1d = []
PV2d = []

lab = tclab.TCLab()

for i in range(0,N):
    t.append(i*Ts)

    # Generate signals SP and DV
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(DVPath,t,DV)

    # Feedforward implementation 
    Delay_RT(DV - DV0*np.ones_like(DV),np.max([thetad-thetap,0]),Ts,MVFFDelay) # ones_likes return an array of ones with the same shape and type as a given array
    LeadLag_RT(MVFFDelay,-Kd/Kp,T1p,T1d,Ts,MVFFLL1)
    LeadLag_RT(MVFFLL1,int(ManFF),T2p,T2d,Ts,MVFF)    # If ActivateFF is false, processe gain will be 0 => int(false) = 0 

    # Manual signals generation : On / Off and values
    SelectPath_RT(ManPath,t,Man)
    SelectPath_RT(MVManPath,t,MVMan)

    # PID control action
    PID_RT(SP,PV,Man,MVMan,MVFF,Kc,Ti,Td,alpha,Ts,MVMin,MVMax,MV,MVP,MVI,MVD,E,ManFF,PV0)

    lab.Q1(MV[-1])
    lab.Q2(DV[-1])
    #MV.append(MVValue)
    #DV.append(DVValue) 

    PV.append(lab.T1);
    

# Save data into a file
now = datetime.now()
date_time = now.strftime("%Y-%m-%d-%Hh%M")
t = np.array(t)
MV = np.array(MV)
PV = np.array(PV)
DV = np.array(DV)
my_data = np.vstack((t.T,MV.T,PV.T,DV.T))
my_data = my_data.T
nameFile = 'Data/Open_loop_experiment_on_' + ExpVariable + '_' + date_time + '.txt'
if not os.path.exists('Data'):
    os.makedirs('Data')
np.savetxt(nameFile,my_data,delimiter=',',header='t,MV,PV,DV',comments='')

IndentationError: unexpected indent (Temp/ipykernel_7056/2471435621.py, line 54)